In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
import instructions

path = os.path.split(os.getcwd())
data_directory = os.path.join(path[0], 'data\\raw')
#data_directory = os.path.join(os.getcwd(), 'data\\raw')


X = instructions.load_csv(data_directory, 'train_data')
y = instructions.load_csv(data_directory, 'train_labels')

In [13]:
from sklearn.preprocessing import binarize

y_binar = binarize(y)
y = pd.DataFrame(np.ravel(y_binar))

In [4]:
from sklearn.preprocessing import StandardScaler

std = StandardScaler()
X_std = std.fit_transform(X)

In [5]:
from sklearn.decomposition import PCA

# Create a PCA that will retain 95% of the variance
pca = PCA(n_components=0.95, whiten=True)

# Conduct PCA
X_pca = pca.fit_transform(X_std)
X_pca = pd.DataFrame(X_pca)

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

pipe = Pipeline(steps=[("classifier", LogisticRegression())])

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier

In [8]:
search_space = [{"classifier" : [LogisticRegression()],
                 "classifier__solver" : ['sag', 'saga', 'liblinear'],
                "classifier__penalty" : ['l1', 'l2', 'elasticnet', 'none'],
                "classifier__C" : [np.logspace(1, 4, 10)],
                "classifier__class_weight" : ['balanced', {"0" : 0.1, "1" : 0.9}]},
                {"classifier" : [BernoulliNB()],
                "classifier__alpha" : [np.linspace(1, 10, 100)]},
                {"classifier" : [RandomForestClassifier()],
                "classifier__n_estimators" : [10, 100, 1000],
                "classifier__criterion" : ['gini', 'entropy', 'log_loss'],
                "classifier__max_features" : ['auto', 'sqrt', 'log2', 'none'],
                "classifier__class_weight" : ['balanced', [{0 : 1}, {1 : 9}]]},
                {"classifier" : [KNeighborsClassifier()],
                "classifier__weights" : ['uniform', 'distance'],
                "classifier__algorithm" : ['auto', 'ball-tree', 'kd-tree', 'brute'],
                "classifier__n_neighbors" : [np.linspace(1, 11, 6)]},
                {"classifier" : [SVC()],
                "classifier__C" : [np.logspace(1, 10000, 1000)],
                "classifier__class_weight" : ['balanced', [{0 : 1}, {1 : 9}]]},
                {"classifier" : [MLPClassifier()]},
                {"classifier" : [DecisionTreeClassifier()],
                "classifier__criterion" : ['gini', 'entropy', 'log_loss'],
                "classifier__splitter" : ['best', 'random'],
                "classifier__max_features" : ['auto', 'sqrt', 'log2', 'none'],
                "classifier__class_weight" : ['balanced', [{0 : 1}, {1 : 9}]]},
                {"classifier" : [ExtraTreesClassifier()],
                "classifier__n_estimators" : [10, 100, 1000],
                "classifier__criterion" : ['gini', 'entropy', 'log_loss'],
                "classifier__max_features" : ['sqrt', 'log2', 'none'],
                "classifier__class_weight" : ['balanced', [{0 : 1}, {1 : 9}]]}]

In [9]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import classification_report

splits = 3
n_repeats = 10
rskf = RepeatedStratifiedKFold(n_splits=splits, n_repeats=n_repeats)

In [10]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import f1_score, balanced_accuracy_score

scoring = ["balanced_accuracy", "f1_weighted"]

search = GridSearchCV(pipe, search_space, cv=rskf, scoring="balanced_accuracy", refit=True, verbose=3)

In [15]:
result = search.fit(X_pca, y)

Fitting 30 folds for each of 208 candidates, totalling 6240 fits
[CV 1/30] END classifier=LogisticRegression(), classifier__C=[   10.            21.5443469     46.41588834   100.
   215.443469     464.15888336  1000.          2154.43469003
  4641.58883361 10000.        ], classifier__class_weight=balanced, classifier__penalty=l1, classifier__solver=sag; balanced_accuracy: (test=nan) f1_weighted: (test=nan) total time=   0.0s
[CV 2/30] END classifier=LogisticRegression(), classifier__C=[   10.            21.5443469     46.41588834   100.
   215.443469     464.15888336  1000.          2154.43469003
  4641.58883361 10000.        ], classifier__class_weight=balanced, classifier__penalty=l1, classifier__solver=sag; balanced_accuracy: (test=nan) f1_weighted: (test=nan) total time=   0.0s
[CV 3/30] END classifier=LogisticRegression(), classifier__C=[   10.            21.5443469     46.41588834   100.
   215.443469     464.15888336  1000.          2154.43469003
  4641.58883361 10000.        ],

c:\Users\Marta\anaconda3\envs\python_in_ml\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 1/30] END classifier=BernoulliNB(), classifier__alpha=[ 1.          1.09090909  1.18181818  1.27272727  1.36363636  1.45454545
  1.54545455  1.63636364  1.72727273  1.81818182  1.90909091  2.
  2.09090909  2.18181818  2.27272727  2.36363636  2.45454545  2.54545455
  2.63636364  2.72727273  2.81818182  2.90909091  3.          3.09090909
  3.18181818  3.27272727  3.36363636  3.45454545  3.54545455  3.63636364
  3.72727273  3.81818182  3.90909091  4.          4.09090909  4.18181818
  4.27272727  4.36363636  4.45454545  4.54545455  4.63636364  4.72727273
  4.81818182  4.90909091  5.          5.09090909  5.18181818  5.27272727
  5.36363636  5.45454545  5.54545455  5.63636364  5.72727273  5.81818182
  5.90909091  6.          6.09090909  6.18181818  6.27272727  6.36363636
  6.45454545  6.54545455  6.63636364  6.72727273  6.81818182  6.90909091
  7.          7.09090909  7.18181818  7.27272727  7.36363636  7.45454545
  7.54545455  7.63636364  7.72727273  7.81818182  7.90909091  8.
  8.09090

c:\Users\Marta\anaconda3\envs\python_in_ml\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 2/30] END classifier=BernoulliNB(), classifier__alpha=[ 1.          1.09090909  1.18181818  1.27272727  1.36363636  1.45454545
  1.54545455  1.63636364  1.72727273  1.81818182  1.90909091  2.
  2.09090909  2.18181818  2.27272727  2.36363636  2.45454545  2.54545455
  2.63636364  2.72727273  2.81818182  2.90909091  3.          3.09090909
  3.18181818  3.27272727  3.36363636  3.45454545  3.54545455  3.63636364
  3.72727273  3.81818182  3.90909091  4.          4.09090909  4.18181818
  4.27272727  4.36363636  4.45454545  4.54545455  4.63636364  4.72727273
  4.81818182  4.90909091  5.          5.09090909  5.18181818  5.27272727
  5.36363636  5.45454545  5.54545455  5.63636364  5.72727273  5.81818182
  5.90909091  6.          6.09090909  6.18181818  6.27272727  6.36363636
  6.45454545  6.54545455  6.63636364  6.72727273  6.81818182  6.90909091
  7.          7.09090909  7.18181818  7.27272727  7.36363636  7.45454545
  7.54545455  7.63636364  7.72727273  7.81818182  7.90909091  8.
  8.09090

...

c:\Users\Marta\anaconda3\envs\python_in_ml\lib\site-packages\sklearn\pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
c:\Users\Marta\anaconda3\envs\python_in_ml\lib\site-packages\sklearn\pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


[CV 28/30] END classifier=ExtraTreesClassifier(), classifier__class_weight=[{0: 1}, {1: 9}], classifier__criterion=log_loss, classifier__max_features=none, classifier__n_estimators=1000; balanced_accuracy: (test=nan) f1_weighted: (test=nan) total time=   0.0s
[CV 29/30] END classifier=ExtraTreesClassifier(), classifier__class_weight=[{0: 1}, {1: 9}], classifier__criterion=log_loss, classifier__max_features=none, classifier__n_estimators=1000; balanced_accuracy: (test=nan) f1_weighted: (test=nan) total time=   0.0s


c:\Users\Marta\anaconda3\envs\python_in_ml\lib\site-packages\sklearn\pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
c:\Users\Marta\anaconda3\envs\python_in_ml\lib\site-packages\sklearn\pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
c:\Users\Marta\anaconda3\envs\python_in_ml\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
4950 fits failed out of a total of 6240.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
---------------

[CV 30/30] END classifier=ExtraTreesClassifier(), classifier__class_weight=[{0: 1}, {1: 9}], classifier__criterion=log_loss, classifier__max_features=none, classifier__n_estimators=1000; balanced_accuracy: (test=nan) f1_weighted: (test=nan) total time=   0.0s


c:\Users\Marta\anaconda3\envs\python_in_ml\lib\site-packages\sklearn\model_selection\_search.py:969: UserWarning: One or more of the test scores are non-finite: [       nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan 0.50019259 0.5        0.5        0.49994074 0.5
 0.5        0.50019259 0.5        0.5               nan        nan
        nan 0.50007407 0.5        0.5        0.5001037  0.5
 0.5        0.50011852 0.5        0.5               nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan     

In [27]:
search.best_score_

AttributeError: 'GridSearchCV' object has no attribute 'best_score_'

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7, test_size = 0.3, shuffle = True, stratify = y)

In [19]:
std = StandardScaler()
X_train_std = std.fit_transform(X_train)
X_test_std = std.transform(X_test)

In [20]:
pca = PCA(n_components=0.95, whiten=True)

# Conduct PCA
X_train_pca = pca.fit_transform(X_train_std)
X_train_pca = pd.DataFrame(X_train_pca)

X_test_pca = pca.transform(X_test_std)
X_test_pca = pd.DataFrame(X_test_pca)

In [21]:
from hpsklearn import HyperoptEstimator, any_classifier, any_preprocessing
from hyperopt import tpe

estim = HyperoptEstimator(classifier=any_classifier("my_clf"),
                          algo=tpe.suggest,
                          max_evals=100,
                          trial_timeout=120)

c:\Users\Marta\anaconda3\envs\python_in_ml\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [22]:
model = estim.fit(X_train_pca, y_train)
model

100%|██████████| 100/100 [00:09<00:00,  9.39s/trial, best loss: 0.10285714285714287]
[17:18:18] WARNING: D:\bld\xgboost-split_1645118015404\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [24]:
print('Score: ', estim.score(X_test_pca, y_test))
print('Best params: ', estim.best_model())

Score:  0.8995555555555556
Best params:  {'learner': XGBClassifier(base_score=0.5, booster='gbtree',
              colsample_bylevel=0.6651992528837498, colsample_bynode=1,
              colsample_bytree=0.6388552251003508, enable_categorical=False,
              gamma=0.006287304974900903, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.0007739000485074648,
              max_delta_step=0, max_depth=10, min_child_weight=5, missing=nan,
              monotone_constraints='()', n_estimators=2800, n_jobs=1,
              num_parallel_tree=1, predictor='auto', random_state=3,
              reg_alpha=0.004462316888033134, reg_lambda=3.92648086586806,
              scale_pos_weight=1, seed=3, subsample=0.8442072722420794,
              tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=None), 'preprocs': (PCA(n_components=4, whiten=True),), 'ex_preprocs': ()}


In [10]:
estim = HyperoptEstimator(classifier=any_classifier("my_clf"),
                          algo=tpe.suggest,
                          max_evals=1000,
                          trial_timeout=120)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y_binar, train_size = 0.7, test_size = 0.3, shuffle = True, stratify = y)

In [12]:
std = StandardScaler()
X_train_std = std.fit_transform(X_train)
X_test_std = std.transform(X_test)

In [13]:
X_train_pca = pca.fit_transform(X_train_std)